In [ ]:
%load_ext autoreload
%autoreload 2

## Installation
If you haven't installed Pearl, please make sure you install Pearl with the following cell. Otherwise, you can skip the cell below.

In [ ]:
%pip uninstall Pearl -y
%rm -rf Pearl
!git clone https://github.com/facebookresearch/Pearl.git
%cd Pearl
%pip install .
%cd ..

## Import Modules

In [ ]:
from pearl.neural_networks.common.value_networks import EnsembleQValueNetwork
from pearl.replay_buffers.sequential_decision_making.bootstrap_replay_buffer import BootstrapReplayBuffer
from pearl.policy_learners.sequential_decision_making.bootstrapped_dqn import BootstrappedDQN
from pearl.utils.functional_utils.experimentation.set_seed import set_seed
from pearl.action_representation_modules.identity_action_representation_module import IdentityActionRepresentationModule
from pearl.history_summarization_modules.lstm_history_summarization_module import LSTMHistorySummarizationModule
from pearl.policy_learners.sequential_decision_making.deep_q_learning import DeepQLearning
from pearl.replay_buffers.sequential_decision_making.fifo_off_policy_replay_buffer import FIFOOffPolicyReplayBuffer
from pearl.utils.functional_utils.train_and_eval.online_learning import online_learning
from pearl.pearl_agent import PearlAgent
from pearl.tutorials.single_item_recommender_system_example.env_model import SequenceClassificationModel
from pearl.tutorials.single_item_recommender_system_example.env import RecEnv
import torch
import matplotlib.pyplot as plt
import numpy as np

set_seed(0)

## Load Environment
This environment's underlying model is trained using the MIND dataset (Wu et al. 2020).

Each data point:
- A history of impressions clicked by a user
- Each impression is represented by an 100-dim vector
- A list of impressions and whether or not they are clicked

The environment is constructed with the following setup. Note that this example is a contrived example to illustrate Pearl's usage, agent modularity and a subset of features. Not to represent a real-world environment or problem.  
- State: a history of impressions by a user (note that we used the history of impressions of instead of clicked impressions to speed up learning in this example. Interested Pearl users can change it to history of clicked impressions with much longer episode length and samples to run the following experiments.)
- Dynamic action space: two randomly picked impressions
- Action: one of the two impressions
- Reward: click
- Reset every 20 steps.


In [ ]:
# load environment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SequenceClassificationModel(100).to(device)
model.load_state_dict(torch.load("Pearl/pearl/tutorials/single_item_recommender_system_example/env_model_state_dict.pt"))
actions = torch.load("Pearl/pearl/tutorials/single_item_recommender_system_example/news_embedding_small.pt")
env = RecEnv(list(actions.values())[:100], model)
observation, action_space = env.reset()

# experiment code
number_of_steps = 100000
record_period = 400

## Vanilla DQN Agent
Able to handle dynamic action space but not able to handle partial observability and sparse reward.

In [ ]:
# create a pearl agent

action_representation_module = IdentityActionRepresentationModule(
    max_number_actions=action_space.n,
    representation_dim=action_space.action_dim,
)

# DQN-vanilla
agent = PearlAgent(
    policy_learner=DeepQLearning(
        state_dim=1,
        action_space=action_space,
        hidden_dims=[64, 64],
        training_rounds=50,
        action_representation_module=action_representation_module,
    ),
    replay_buffer=FIFOOffPolicyReplayBuffer(100_000),
    device_id=-1,
)

info = online_learning(
    agent=agent,
    env=env,
    number_of_steps=number_of_steps,
    print_every_x_steps=100,
    record_period=record_period,
    learn_after_episode=True,
)
torch.save(info["return"], "DQN-return.pt")
plt.plot(record_period * np.arange(len(info["return"])), info["return"], label="DQN")
plt.legend()
plt.show()

## DQN Agent with LSTM history summarization module

Now the DQN agent can handle partially observable environments with history summarization

In [ ]:
# Add a LSTM history summarization module

agent = PearlAgent(
    policy_learner=DeepQLearning(
        state_dim=128,
        action_space=action_space,
        hidden_dims=[64, 64],
        training_rounds=50,
        action_representation_module=action_representation_module,
    ),
    history_summarization_module=LSTMHistorySummarizationModule(
        observation_dim=1,
        action_dim=100,
        hidden_dim=128,
        history_length=8,
    ),
    replay_buffer=FIFOOffPolicyReplayBuffer(100_000),
    device_id=-1,
)

info = online_learning(
    agent=agent,
    env=env,
    number_of_steps=number_of_steps,
    print_every_x_steps=100,
    record_period=record_period,
    learn_after_episode=True,
)
torch.save(info["return"], "DQN-LSTM-return.pt")
plt.plot(record_period * np.arange(len(info["return"])), info["return"], label="DQN-LSTM")
plt.legend()
plt.show()

## Bootstrapped DQN Agent with LSTM History Summarization

Leveraging the deep exploration value-based algorithm, now the agent can achieve a better performance in a much faster way while being able to still leverage history summarization capability. 

In [ ]:
# Better exploration with BootstrappedDQN-LSTM

agent = PearlAgent(
    policy_learner=BootstrappedDQN(
        q_ensemble_network=EnsembleQValueNetwork(
            state_dim=128,
            action_dim=100,
            ensemble_size=10,
            output_dim=1,
            hidden_dims=[64, 64],
            prior_scale=0.3,
        ),
        action_space=action_space,
        training_rounds=50,
        action_representation_module=action_representation_module,
    ),
    history_summarization_module=LSTMHistorySummarizationModule(
        observation_dim=1,
        action_dim=100,
        hidden_dim=128,
        history_length=8,
    ),
    replay_buffer=BootstrapReplayBuffer(100_000, 1.0, 10),
    device_id=-1,
)

info = online_learning(
    agent=agent,
    env=env,
    number_of_steps=number_of_steps,
    print_every_x_steps=100,
    record_period=record_period,
    learn_after_episode=True,
)
torch.save(info["return"], "BootstrappedDQN-LSTM-return.pt")
plt.plot(record_period * np.arange(len(info["return"])), info["return"], label="BootstrappedDQN-LSTM")
plt.legend()
plt.show()

## Summary
In this example, we illustrated Pearl's capability of dealing with dynamic action space, standard policy learning, history summarization and intelligent exploration, all in a single agent. By running the code above, you should be able to get agent performance results similar to the figure shown in pearl/tutorials/single_item_recommender_system_example/dqn+lstm+deep_explore.png.
